# Phase 6: Model Optimization & Hyperparameter Tuning
Fine-tune the best performing models and create ensemble methods

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

## Option 1: Quick Optimization (Recommended)
Run automated hyperparameter tuning for traditional ML models

In [ ]:
# This will take 10-20 minutes depending on your settings
# Set tune_svm_flag = False to skip SVM tuning (saves ~20 minutes)
%run ../src/optimization/hyperparameter_tuning.py

## Option 2: Manual Hyperparameter Tuning
For more control over the optimization process

### 2.1 Load Best Model Results

In [ ]:
# Load evaluation results to see which model performed best
df_results = pd.read_csv('../results/model_comparison.csv')
print("Top 3 Models:")
display(df_results.head(3))

### 2.2 Load Features

In [ ]:
from src.optimization.hyperparameter_tuning import load_features

X_train, y_train, X_val, y_val, X_test, y_test = load_features('tfidf')

print(f"Train set: {X_train.shape}")
print(f"Val set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

### 2.3 Tune Logistic Regression

In [ ]:
from src.optimization.hyperparameter_tuning import tune_logistic_regression

lr_model, lr_params = tune_logistic_regression(
    X_train, y_train, X_val, y_val, 
    search_type='grid'  # or 'random'
)

### 2.4 Tune Random Forest

In [ ]:
from src.optimization.hyperparameter_tuning import tune_random_forest

rf_model, rf_params = tune_random_forest(
    X_train, y_train, X_val, y_val,
    search_type='random'  # Random search is faster for RF
)

### 2.5 Create Ensemble Model

In [ ]:
from src.optimization.hyperparameter_tuning import create_ensemble_model

# Combine best models
models_dict = {
    'logistic': lr_model,
    'random_forest': rf_model
}

ensemble_model, voting_type = create_ensemble_model(
    models_dict, X_train, y_train, X_val, y_val
)

### 2.6 Evaluate on Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate ensemble
y_pred = ensemble_model.predict(X_test)

print("\nENSEMBLE MODEL - TEST SET PERFORMANCE")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Fake', 'Real']))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Fake', 'Real'],
           yticklabels=['Fake', 'Real'])
plt.title('Ensemble Model - Confusion Matrix', fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## Option 3: Deep Learning Model Optimization
Fine-tune LSTM, BiLSTM, or CNN-LSTM models

In [ ]:
from src.optimization.hyperparameter_tuning import optimize_deep_learning_model

# Choose model type: 'lstm', 'bilstm', or 'cnn_lstm'
model_type = 'lstm'  # Change this

# This will take 30-60 minutes depending on GPU
best_model, best_params, tuning_results = optimize_deep_learning_model(
    model_type=model_type,
    epochs_range=[10, 15, 20],
    batch_sizes=[64, 128],
    learning_rates=[0.001, 0.0005]
)

print("\nTuning Results:")
display(tuning_results.head(10))

### Visualize Hyperparameter Impact

In [ ]:
# Plot learning rate vs F1 score
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
tuning_results.groupby('learning_rate')['val_f1'].mean().plot(kind='bar')
plt.title('Learning Rate Impact')
plt.ylabel('Validation F1')

plt.subplot(1, 3, 2)
tuning_results.groupby('batch_size')['val_f1'].mean().plot(kind='bar')
plt.title('Batch Size Impact')
plt.ylabel('Validation F1')

plt.subplot(1, 3, 3)
tuning_results.groupby('epochs')['val_f1'].mean().plot(kind='bar')
plt.title('Epochs Impact')
plt.ylabel('Validation F1')

plt.tight_layout()
plt.show()

## Comparison: Before vs After Optimization

In [ ]:
# Load original results
original_results = pd.read_csv('../results/model_comparison.csv')

# You can manually add optimized results here
print("BEFORE OPTIMIZATION:")
display(original_results.head())

# After running optimization, compare results

## Save Optimized Models

In [ ]:
import pickle
import os

# Save ensemble model
if 'ensemble_model' in locals():
    with open('../models/ensemble_optimized.pkl', 'wb') as f:
        pickle.dump(ensemble_model, f)
    print("✅ Ensemble model saved!")

# Save individual optimized models
if 'lr_model' in locals():
    with open('../models/logistic_optimized.pkl', 'wb') as f:
        pickle.dump(lr_model, f)
    print("✅ Optimized Logistic Regression saved!")

if 'rf_model' in locals():
    with open('../models/random_forest_optimized.pkl', 'wb') as f:
        pickle.dump(rf_model, f)
    print("✅ Optimized Random Forest saved!")

# Save best params
os.makedirs('../results/optimization', exist_ok=True)
if 'lr_params' in locals():
    with open('../results/optimization/lr_best_params.pkl', 'wb') as f:
        pickle.dump(lr_params, f)

if 'rf_params' in locals():
    with open('../results/optimization/rf_best_params.pkl', 'wb') as f:
        pickle.dump(rf_params, f)

## Summary & Next Steps

In [ ]:
print("""
═══════════════════════════════════════════════════════════════════════════════
                    PHASE 6: OPTIMIZATION COMPLETE!
═══════════════════════════════════════════════════════════════════════════════

✅ Completed Tasks:
   • Hyperparameter tuning for traditional ML models
   • Created ensemble models with voting classifiers
   • Optimized best performing models
   • Saved optimized models for deployment

📊 Results Saved:
   • Optimized models: models/
   • Best parameters: results/optimization/
   • Tuning results: results/*_tuning_results.csv

🚀 NEXT STEP: Phase 7 - Deployment
   • Create Flask/FastAPI web application
   • Build interactive UI for predictions
   • Containerize with Docker
   • Deploy to cloud (Heroku/AWS/GCP)

═══════════════════════════════════════════════════════════════════════════════
""")